<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/paraphrase_%EC%8B%A4%ED%97%98%EC%8B%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시작

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/파이널 프로젝트/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 추가 import

In [ ]:
# 경고출력 무시
import warnings
warnings.filterwarnings('ignore')

# 하이퍼파라미터 튜닝용
!pip install optuna
import optuna

# 트랜스포머
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00


# 함수들 정리

# 데이터 전처리

In [ ]:
# yogi_df = pd.read_csv(f'{DATA_PATH}yogiyo_reviews_jsi.csv')

# yogi_df.shape

(18686, 6)

In [ ]:
# yogi_df.head()

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,5,5,5,5.0,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


## 중복제거

In [ ]:
# yogi_df.drop_duplicates(subset='사장댓글', keep=False, ignore_index=True, inplace=True)

# yogi_df.shape

(10502, 6)

## 정규표현식

In [ ]:
# yogi_df['고객리뷰'] = yogi_df.고객리뷰.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)
# yogi_df.고객리뷰

0                         리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!
1                       잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
2                                       굿 말이 필요 없는 메뉴 였습니다
3                                              맛있어요 양도많구좋음
4                                                양많고 맛있었어요
                               ...                        
10497    사장님 넘 맛잇어요전화해서 양념두 더 달라고 햇엇는데 친절하게 대해주시고 감사합니다...
10498                                   잘먹었숩당 양은 증말로 많아요!!
10499                홍대개미넘 마싯어요부산 놀러가서 첫끼가 홍대개미였는대 참순삭했습니다
10500                             밥에 비해 양이 적어요  특히 연어는,,,,
10501            요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다
Name: 고객리뷰, Length: 10502, dtype: object

In [ ]:
# # 정규표현식
# yogi_df['사장댓글'] = yogi_df.사장댓글.str.replace('[^a-zA-Z가-힣0-9 .,!?\'\"]', '', regex=True)

# # 개인ID -> 고객님 변경
# yogi_df['사장댓글'] = yogi_df.사장댓글.str.replace("\w+님", "고객님", regex=True)
# yogi_df['사장댓글'] = yogi_df.사장댓글.str.replace("\w+ 님", "고객님", regex=True)

# yogi_df.사장댓글

0        좋은 리뷰 덕분에 기분이 행복해지는 것 고객님께서 맛있다고 말씀 주시니 피로가 싹 ...
1        소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
2        수 많은 매장들을 헤치고 와주셔서 감사합니다!!저희 매장의 메뉴를 제대로 즐기신 거...
3        수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
4        저희 매장 찾아주신 것도 감사한데 리뷰까지..고객님들의 배고픔을 저희는 지켜만 볼 ...
                               ...                        
10497    요청사항 언제든 환영입니다 맛있게 드셔주셔서 감사합니다!!리뷰 감사합니다!!!다음에...
10498                 맛있게 드셔주셔서 감사합니다!별 5개 받도록 노력하겠습니다....
10499                                헉 부산에서 처음..!!! 감사합니당 
10500      아마 두툼하게 썰려서 그런것 같습니다  다음에도 이용해주신다면 만족시켜드리겠습니다!!
10501                                죄송합니다 !! 다음에도 애용해주세용 
Name: 사장댓글, Length: 10502, dtype: object

In [ ]:
# # 불필요한 문자 제거

# # 고객리뷰 부분
# content_list = []
# for content in yogi_df['고객리뷰']:
#     if isinstance(content, str):
#         c_result = re.sub(r'\.{2,}', '.', content)
#         c_result = re.sub(r'\,{2,}', ',', c_result)
#         c_result = re.sub(r'\~{2,}', '~', c_result)
#         c_result = re.sub(r'\!{2,}', '!', c_result)
#         c_result = re.sub(r'\?{2,}', '?', c_result)
#         c_result = re.sub(' +', ' ', c_result)
#         c_result = c_result.lstrip()

#         content_list.append(c_result)
#     else:
#         content_list.append('')

# yogi_df['고객리뷰'] = content_list

# # 사장댓글 부분
# replyContent_list = []
# for replyContent in yogi_df['사장댓글']:
#     if isinstance(replyContent, str):
#         r_result = re.sub(r'\.{2,}', '.', replyContent)
#         r_result = re.sub(r'\,{2,}', ',', r_result)
#         r_result = re.sub(r'\~{2,}', '~', r_result)
#         r_result = re.sub(r'\!{2,}', '!', r_result)
#         r_result = re.sub(r'\?{2,}', '?', r_result)
#         r_result = re.sub(r' +', ' ', r_result)
#         r_result = r_result.lstrip()

#         replyContent_list.append(r_result)
#     else:
#         replyContent_list.append('')

# yogi_df['사장댓글'] = replyContent_list

# yogi_df.head()

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
0,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 고객님께서 맛있다고 말씀 주시니 피로가 싹 ...
1,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
2,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!저희 매장의 메뉴를 제대로 즐기신 거 ...
3,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
4,양많고 맛있었어요,5,5,5,5.0,저희 매장 찾아주신 것도 감사한데 리뷰까지.고객님들의 배고픔을 저희는 지켜만 볼 수...


## 가게명 제거 함수

In [ ]:
# # 가게명 패턴 함수들
# def text_ver_1():
#     for i in range(yogi_df['사장댓글'].shape[0]):
#         pat = '저희\s+'
#         tmp = re.split(pat, yogi_df['사장댓글'][i])
#         if len(tmp) == 1:
#             # print(f'{i}번 "저희" 없어서 패스')
#             pass
#         else:
#             tmp = re.split('의', tmp[1])
#             cnt = 0
#             for t in tmp:
#                 if len(t) == 0:
#                     pass
#                 elif t[0] == ' ':
#                     # print(f'{i}번 수정완료')
#                     break
#                 elif t[0] == '매':
#                     # print(f'{i}번 "가게명" 없어서 패스')
#                     break
#                 else:
#                     t = t + '의'
#                     yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')
#                     cnt += 1
#                     if cnt == len(tmp):
#                         # print(f'{i}번 수정부분 없음')
#                         pass

#     print('text_ver_1 실행완료')


# def text_ver_2():
#     for i in range(yogi_df['사장댓글'].shape[0]):
#         pat = '안녕하세요\s+'
#         tmp = re.split(pat, yogi_df['사장댓글'][i])
#         if len(tmp) == 1:
#             # print(f'{i}번 "안녕하세요" 없어서 패스')
#             pass
#         else:
#             tmp = re.split('입니다', tmp[1])
#             cnt = 0
#             for t in tmp:
#                 if len(t) == 0:
#                     pass
#                 elif t[0] == ' ':
#                     # print(f'{i}번 수정완료')
#                     break
#                 else:
#                     t = t + '입니다'
#                     yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')
#                     cnt += 1
#                     if cnt == len(tmp):
#                         # print(f'{i}번 수정부분 없음')
#                         pass

#     print('text_ver_2 실행완료')


# def text_ver_3():
#     for i in range(yogi_df['사장댓글'].shape[0]):
#         pat = '안녕하세요.\s+'
#         tmp = re.split(pat, yogi_df['사장댓글'][i])
#         if len(tmp) == 1:
#             # print(f'{i}번 "안녕하세요." 없어서 패스')
#             pass
#         else:
#             tmp = re.split('입니다.', tmp[1])
#             cnt = 0
#             for t in tmp:
#                 if len(t) == 0:
#                     pass
#                 elif t[0] == ' ':
#                     # print(f'{i}번 수정완료')
#                     break
#                 else:
#                     t = t + '입니다.'
#                     yogi_df['사장댓글'][i] = yogi_df['사장댓글'][i].replace(t, '')
#                     cnt += 1
#                     if cnt == len(tmp):
#                         # print(f'{i}번 수정부분 없음')
#                         pass

#     print('text_ver_3 실행완료')


# # 패턴 함수들 실행시킬 함수
# def text_remove(text):
#     if text == '저희':
#         return text_ver_1()
#     elif text == '안녕하세요':
#         return text_ver_2()
#     elif text == '안녕하세요.':
#         return text_ver_3()
#     else:
#         print(f'"{text}"는 실행할 수 없는 단어입니다.')

In [ ]:
# check_lst = ['저희', '안녕하세요', '안녕하세요.']

# for lst in check_lst:
#     text_remove(lst)

text_ver_1 실행완료
text_ver_2 실행완료
text_ver_3 실행완료


## 컬럼 이름 변경

In [ ]:
# tmp = {
#     'content': yogi_df['고객리뷰'],
#     'score': yogi_df['별점'],
#     'replycontent': yogi_df['사장댓글']
# }

# yogi_df = pd.DataFrame(tmp)
# yogi_df

,content,score,replycontent
0,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,5,좋은 리뷰 덕분에 기분이 행복해지는 것 고객님께서 맛있다고 말씀 주시니 피로가 싹 ...
1,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
2,굿 말이 필요 없는 메뉴 였습니다,5,수 많은 매장들을 헤치고 와주셔서 감사합니다!저희 매장의 메뉴를 제대로 즐기신 거 ...
3,맛있어요 양도많구좋음,5,수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
4,양많고 맛있었어요,5,저희 매장 찾아주신 것도 감사한데 리뷰까지.고객님들의 배고픔을 저희는 지켜만 볼 수...
...,...,...,...
10497,사장님 넘 맛잇어요전화해서 양념두 더 달라고 햇엇는데 친절하게 대해주시고 감사합니다...,5,요청사항 언제든 환영입니다 맛있게 드셔주셔서 감사합니다!리뷰 감사합니다!다음에 또 ...
10498,잘먹었숩당 양은 증말로 많아요!,4,맛있게 드셔주셔서 감사합니다!별 5개 받도록 노력하겠습니다.
10499,홍대개미넘 마싯어요부산 놀러가서 첫끼가 홍대개미였는대 참순삭했습니다,5,헉 부산에서 처음.! 감사합니당
10500,"밥에 비해 양이 적어요 특히 연어는,",3,아마 두툼하게 썰려서 그런것 같습니다 다음에도 이용해주신다면 만족시켜드리겠습니다!


## 맞춤법

In [ ]:
# !git clone https://github.com/ssut/py-hanspell.git

Cloning into 'py-hanspell'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 110 (delta 16), reused 9 (delta 9), pack-reused 88
Receiving objects: 100% (110/110), 22.30 KiB | 7.43 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
# !cd py-hanspell

In [ ]:
# !pip install py-hanspell/

Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=5920703b35836de5c7755954f84ef697418ce6e3c49cb439e7f9f17bc6d15aba
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell


In [ ]:
# from hanspell import spell_checker

# def correct_spell(text):
#     if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
#         corrected_text = spell_checker.check(text).checked
#         return corrected_text
#     else:
#         return text

In [ ]:
# # 고객리뷰 검사
# for i in tqdm(range(yogi_df['content'].shape[0])):
#     yogi_df['content'][i] = correct_spell(yogi_df['content'][i])

# # 사장댓글 검사
# for i in tqdm(range(yogi_df['replycontent'].shape[0])):
#     yogi_df['replycontent'][i] = correct_spell(yogi_df['replycontent'][i])

In [ ]:
# # 전처리 완료된 데이터 저장
# yogi_df.to_csv(f"{DATA_PATH}preprocessed_yogi_df.csv")
# yogi_df.to_excel(f"{DATA_PATH}preprocessed_yogi_df.xlsx")

# 전처리된 데이터

In [ ]:
yogi_df = pd.read_csv(f'{DATA_PATH}preprocessed_yogi_df.csv')

yogi_df.shape

# Paraphrase

```shell
https://huggingface.co/lcw99/t5-base-korean-paraphrase
```

In [ ]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

## 실험

In [ ]:
prompt = yogi_df['replycontent'][34]
prompt

'1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다. 저희 매장에서는 배달로 드릴 때 배달 시간을 지키기 위해 최대한 신속하고 정확하게 조리를 하고 있습니다. 다만 배달 대행사를 이용하고 있어서, 메뉴를 전달드리고 나서부터는 실시간으로 파악하기에 어려운 부분이 있는 거 같습니다. 지연에 관한 내용은 대행사측에 전달하여, 추후에는 일어나지 않도록 노력하겠습니다. 다음에는 이번보다 더 온전한 모습으로 만나 뵐 수 있도록 힘내겠습니다. 감사합니다. '

In [ ]:
generated_text = pipe(prompt,
                             max_length = 128,
                             num_beams = 5,
                             num_return_sequences = 4,
                             no_repeat_ngram_size = 4,
                             top_k = 81,
                             top_p = 0.8288,
                             early_stopping=True)
# print("리뷰: ", prompt)
print(generated_text[0]['generated_text'])

저희 매장은 배달 전문 업체로 고객님의 요청이 있을 경우 신속한 처리를 위해 매장 내에 비치된 냉장고를 이용하고 있는데, 시간이 초과될 경우 고객님께 바로 알려드릴 수 있도록 하겠습니다.


In [ ]:
(generated_text[0]['generated_text'])

'양이 2만원씩이나하는데 진짜 장난하나 싶어서 글을 썼는데 절대 제가 돈주고 사서 먹을 정도는 아닙니다 진짜'

## 하이퍼파라미터 튜닝

In [ ]:
yogi_df.score.value_counts()

5    9776
4     422
3     153
1      90
2      61
Name: score, dtype: int64

In [ ]:
score_1 = yogi_df[yogi_df['score'] == 1]
score_2 = yogi_df[yogi_df['score'] == 2]
score_3 = yogi_df[yogi_df['score'] == 3]
score_4 = yogi_df[yogi_df['score'] == 4]

score_df = pd.concat([score_1, score_2, score_3, score_4], ignore_index=True)
score_df.head()

,content,score,replyContent
0,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 ...,1,안녕하세요 .주신 리뷰 잘 읽었습니다. 뒤돌아 보니 저희 응대에 미흡했던 점이 있던...
1,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고,1,"솔직한 리뷰 남겨 주셔서 감사합니다. 저희 견 전달 해서, 더 개선 될 수 있는 부..."
2,만들어 놓은지 한참 된 커피 .배달 예정 시간 25분이 지났어요.,1,안녕하세요 배달이 늦어져 죄송합니다. 한파로 길이 얼어서 고객님들이 상황이 여유치가...
3,밥이 쉬었어요. 단무지 때문도 아니고 김이 많아서 그런것도 아니고 쉬어서 냄새가 나...,1,밥이 쉬다뇨 말도 안되는 소리좀 하지마세요. 진짜 먼 정신이 이상한 사람인가 하루에...
4,잠실점에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와보니...,1,요기요가 주문어플에 렉이 자주 있어서. 여러가지로 혼돈이 있어 정말 죄송하게 되었습...


In [ ]:
score_df.score.value_counts()

4    422
3    153
1     90
2     61
Name: score, dtype: int64

### BLEU 계산

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
# BLEU값은 0이 가장 작고, 1이 가장 큰 값입니다.
tmp_score1 = sentence_bleu(['I was happy'], 'I was happy')
tmp_score2 = sentence_bleu(['I was happy'], 'I happy was')

print(tmp_score1)
print(tmp_score2)

1.0
0.7400828044922853


In [ ]:
def calculate_bleu(text, generated_text):
    reference_sentence = text # 실제데이터 = 정답데이터
    if type(generated_text) == list:
        generated_sentence = generated_text[0]['generated_text'] # 모델이 생성한 데이터
    else:
        generated_sentence = generated_text # 모델이 생성한 데이터

    score = sentence_bleu([reference_sentence], generated_sentence)
    return score

In [ ]:
calculate_bleu('I was happy', 'I was happy')

1.0

In [ ]:
prompt = score_df['content'][0]
generated_text = pipe(prompt, max_length=128, num_beams=3, num_return_sequences=1, no_repeat_ngram_size=3, top_k=10, top_p=0.8, early_stopping=True)
print("리뷰: ", prompt)
print('변환: ', generated_text[0]['generated_text'])

리뷰:  리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 전화해서 배달이 안온다 했더니 여자분께서 한숨 푹 쉬시면서 하아비가 와서 배달 기사 안 잡힌다 하시는데 제가 뭐 그런거에 이해 못하는 사람도 아니고 그냥 안와서 물어보는건데 한숨음 왜 크게 쉬시나요? 아니 돈을 내고 배달 시간도 20분이나 지나서 물어보는게 잘못된건가요? 배달기사한테 도착하면 문자 달라고 하니까 문자도 안주셔서 밥은 식은 상태로 먹고 진짜 배달 시켜 먹어본곳 중에 제일 최악이네요 여긴
변환:  제가 아무리 생각해도 이해가 안 가는 게 배달 오기를 20분 넘게 기다렸고 문자도 한 번 못 받았는데 배달기사님 댁에 도착하면 문자로 다시 와 달라고 할 정도니까요.


In [ ]:
calculate_bleu(prompt, generated_text)

0.05812550410840367

### 고객리뷰 부분

In [ ]:
from torch.cuda import is_available

def objective(trial):
    hp = {
        'num_beams': trial.suggest_int('num_beams', 1, 10),
        'num_return_sequences': trial.suggest_int('num_return_sequences', 1, 5),
        'no_repeat_ngram_size': trial.suggest_int('no_repeat_ngram_size', 1, 5),
        'top_k': trial.suggest_int('top_k', 10, 100),
        'top_p': trial.suggest_float('top_p', 0.8, 1.0),
    }

    # num_return_sequences가 num_beams보다 작거나 같아야 하기 때문에 조건문 추가
    if hp["num_return_sequences"] > hp["num_beams"]:
        hp["num_return_sequences"] = hp["num_beams"]

    pipe = pipeline(
        "text2text-generation",
        model="lcw99/t5-base-korean-paraphrase",
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        )

    bleu_score_lst = []
    for review in tqdm(score_df['content']): # 별점 1~4까지만
        generated_text = pipe(review, max_length=128, **hp, early_stopping=True)

        bleu_score = calculate_bleu(review, generated_text)
        bleu_score_lst.append(bleu_score)

    bleu_score_mean = sum(bleu_score_lst) / len(bleu_score_lst)
    return bleu_score_mean

In [ ]:
# sampler = optuna.samplers.TPESampler(seed=SEED)
# study = optuna.create_study(
#     direction = 'maximize', # maximize, minaize
#     sampler = sampler
# )
# study.optimize(objective, n_trials=15)

In [ ]:
study.best_params

# ’num_beams’: 10,
# ‘num_return_sequences’: 4,
# ‘no_repeat_ngram_size’: 4,
# ‘top_k’: 11,
# ‘top_p’: 0.8033

In [ ]:
study.best_value

# 0.2861

### 사장댓글 부분

In [ ]:
from torch.cuda import is_available

def objective(trial):
    hp = {
        'num_beams': trial.suggest_int('num_beams', 1, 10),
        'num_return_sequences': trial.suggest_int('num_return_sequences', 1, 5),
        'no_repeat_ngram_size': trial.suggest_int('no_repeat_ngram_size', 1, 5),
        'top_k': trial.suggest_int('top_k', 10, 100),
        'top_p': trial.suggest_float('top_p', 0.8, 1.0),
    }

    # num_return_sequences가 num_beams보다 작거나 같아야 하기 때문에 조건문 추가
    if hp["num_return_sequences"] > hp["num_beams"]:
        hp["num_return_sequences"] = hp["num_beams"]

    pipe = pipeline(
        "text2text-generation",
        model="lcw99/t5-base-korean-paraphrase",
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        )

    bleu_score_lst_2 = []
    for review in tqdm(score_df['replycontent']):
        generated_text = pipe(review, max_length=128, **hp, early_stopping=True)

        bleu_score_2 = calculate_bleu(review, generated_text)
        bleu_score_lst_2.append(bleu_score_2)

    bleu_score_mean_2 = sum(bleu_score_lst_2) / len(bleu_score_lst_2)
    return bleu_score_mean_2

In [ ]:
# sampler = optuna.samplers.TPESampler(seed=SEED)
# study = optuna.create_study(
#     direction = 'maximize', # maximize, minaize
#     sampler = sampler
# )
# study.optimize(objective, n_trials=15)

In [ ]:
# study.best_params

# ’num_beams’: 5,

# ‘num_return_sequences’: 4,

# ‘no_repeat_ngram_size’: 4,

# ‘top_k’: 81,

# ‘top_p’: 0.8

In [ ]:
# study.best_value

# 0.1953

## 리뷰 paraphrase

In [ ]:
# 변환 결과 덮어쓸 임시 데이터프레임 준비
tmp_df = yogi_df.copy()
tmp_df.head()

,content,score,replycontent
0,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,5,좋은 리뷰 덕분에 기분이 행복해지는 것 고객님께서 맛있다고 말씀 주시니 피로가 싹 ...
1,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
2,굿 말이 필요 없는 메뉴 였습니다,5,수 많은 매장들을 헤치고 와주셔서 감사합니다!저희 매장의 메뉴를 제대로 즐기신 거 ...
3,맛있어요 양도많구좋음,5,수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
4,양많고 맛있었어요,5,저희 매장 찾아주신 것도 감사한데 리뷰까지.고객님들의 배고픔을 저희는 지켜만 볼 수...


### 고객리뷰 부분

In [ ]:
for i in tqdm(range(yogi_df.shape[0])):
    generated_content = pipe(yogi_df[f'content'][i],
                             max_length = 128,
                             num_beams = 10,
                             num_return_sequences = 4,
                             no_repeat_ngram_size = 4,
                             top_k = 11,
                             top_p = 0.8033,
                             early_stopping=True)

    tmp_df[f'content'][i] = generated_content[0]['generated_text']
    break

  0%|          | 0/10502 [00:00<?, ?it/s]

### 사장댓글 부분

In [ ]:
for i in tqdm(range(yogi_df.shape[0])):
    generated_replycontent = pipe(yogi_df[f'replycontent'][i],
                             max_length = 128,
                             num_beams = 5,
                             num_return_sequences = 4,
                             no_repeat_ngram_size = 4,
                             top_k = 81,
                             top_p = 0.8288,
                             early_stopping=True)

    tmp_df[f'replycontent'][i] = generated_replycontent[0]['generated_text']
    break

  0%|          | 0/10502 [00:00<?, ?it/s]

In [ ]:
tmp_df['replycontent'].head()

0    어려운 일이 주방에서 벌어지고 있는 가운데 매번 손보기가 쉽지 않지만 이런 좋은 글...
1    소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
2    수 많은 매장들을 헤치고 와주셔서 감사합니다!저희 매장의 메뉴를 제대로 즐기신 거 ...
3    수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
4    저희 매장 찾아주신 것도 감사한데 리뷰까지.고객님들의 배고픔을 저희는 지켜만 볼 수...
Name: replycontent, dtype: object

# 오버샘플링

## 저장된 csv파일 불러오기

In [ ]:
train_over = pd.read_csv(f'{DATA_PATH}train_over.csv')
train_over

,review,reply,star
0,아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서...,저희가 미처 생각하지 못한 부분이 있기 때문에 다시 한번 감사의 마음을 전하고 싶습니다.,1
1,양이 2만원씩이나하는데 장난으로하는것도 아니고 진짜 이거 2만원주고 사서 먹을 정도...,저희 1988응답하라추억의옛날도시락은 언제나 약속 드린 정량을 지키기 위해 노력하고...,1
2,커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만...,한파로 길이 꽁꽁 얼어 고객님들이 불편을 드려 죄송했는데 주문해 주셔서 감사합니다.,1
3,김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었...,이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 ...,1
4,처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역...,잠실점에서 기분이 상하셨었는데 저희가 다시 실수를 한 것 같은데 다시 한번 연락드리...,1
...,...,...,...
721,맛있게 잘 먹었습니다.,더 노력하는 보둘미역이 되도록 최선을 다해 보겠습니다 맛있는 음식 만들어 주셔서 감...,4
722,시원한 국물이 더 많았으면 국물 맛은 더 시원했을 것 같아요 시원한 국을 시원하게 ...,배달용기에 국물을 더 부우시면 국물이 흘러 내리기 때문에 혹시라도 모자라면 댁에서 ...,4
723,맛있어요 포장도 꼼꼼하고 무엇보다 반찬이 짜지 않고 맛있어요 구성이 여러 가지라 더...,보통은 1인분에 2인분 정도 먹는 것으로 알고 있는데 많은 분들이 그렇게 적게 주셨...,4
724,양이 진짜 많아요 진짜 많이 많이 덕분에 맛있게 먹었어요,맛있게 잘 먹었고 별 다섯 개 드리도록 하겠습니다 맛있게 먹어 주셔서 감사합니다,4


In [ ]:
train_over.head()

,review,reply,star
0,아니 그런데 배달이 20분 넘게 안 와서 전화해서 물었더니 배달 기사님이 비가 와서...,저희가 미처 생각하지 못한 부분이 있기 때문에 다시 한번 감사의 마음을 전하고 싶습니다.,1
1,양이 2만원씩이나하는데 장난으로하는것도 아니고 진짜 이거 2만원주고 사서 먹을 정도...,저희 1988응답하라추억의옛날도시락은 언제나 약속 드린 정량을 지키기 위해 노력하고...,1
2,커피가 배달되기를 기다린 지 25분이 지나자 커피 배달 예정 시간은 이미 커피를 만...,한파로 길이 꽁꽁 얼어 고객님들이 불편을 드려 죄송했는데 주문해 주셔서 감사합니다.,1
3,김이 많아서 그런것도 있고 단무짠지 김이 많이 없어서 그런지도 모르겠지만 밥이 쉬었...,이런 댓글은 본인 가족이나 지인에게 하는데 본인이 아니더라도 주변 사람이 해야 할 ...,1
4,처음 먹었을 때 너무 맛있게 먹었던 기억에 다시 찾아 주문하려고 잠실점에 갔는데 역...,잠실점에서 기분이 상하셨었는데 저희가 다시 실수를 한 것 같은데 다시 한번 연락드리...,1


In [ ]:
train_over.star.value_counts()

4    422
3    153
1     90
2     61
Name: star, dtype: int64

In [ ]:
model_name = 'MLOpsEngineer/review-rate-prediction'

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00


In [ ]:
text = yogi_df['고객리뷰'][0]
text, type(text)

('사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~', str)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tmp = tokenizer(text, truncation=True, padding='max_length')
tmp

{'input_ids': [2, 41618, 38818, 4374, 8232, 15482, 2070, 16251, 17143, 18480, 2070, 17595, 33486, 3444, 18877, 38044, 2070, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
tmp.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train = tokenizer(yogi_df['고객리뷰'].tolist(), truncation=True, padding='max_length',return_tensors="pt")
train

{'input_ids': tensor([[    2, 41618, 38818,  ...,     0,     0,     0],
        [    2, 37065,  3067,  ...,     0,     0,     0],
        [    2,  5957, 22796,  ...,     0,     0,     0],
        ...,
        [    2, 17595, 24835,  ...,     0,     0,     0],
        [    2,  5957, 18773,  ...,     0,     0,     0],
        [    2,  2006,  5223,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
train_input_ids = np.array(train['input_ids'])
train_token_type_ids = np.array(train['token_type_ids'])
train_attention_mask = np.array(train['attention_mask'])

train_input_ids.shape, train_token_type_ids.shape, train_attention_mask.shape

((18686, 512), (18686, 512), (18686, 512))

In [ ]:
tmp = yogi_df['별점'].tolist()

for i in range(len(tmp)):
    tmp[i] = int(tmp[i])

target = np.array(tmp)
target, target.dtype, target.shape

(array([5, 5, 5, ..., 5, 5, 5]), dtype('int64'), (18686,))

In [ ]:
target.tolist()

[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 3,
 2,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 5,
 4,
 5,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 5,
 4,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 3,
 3,
 3,
 3,
 5,
 5,
